# 7. LangSmith を使った RAG アプリケーションの評価


In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), 'rag_ai_agent_book', '.env')
load_dotenv(dotenv_path)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "rag-ai-agent-book"

## 7.4. Ragas による合成テストデータの生成


### パッケージのインストール


In [2]:
# !pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
#     langchain-community==0.2.12 GitPython==3.1.43 \
#     langchain-chroma==0.1.2 chromadb==0.5.3 \
#     ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.10.6

### 検索対象のドキュメントのロード


In [7]:
from langchain_community.document_loaders import GitLoader  # Gitリポジトリからドキュメントをロードするためのクラスをインポート

# .mdxファイルのみを対象とするフィルタ関数を定義
def file_filter(file_path: str) -> bool:
    # ファイルパスが.mdxで終わる場合のみTrueを返す
    return file_path.endswith(".mdx")

# GitLoaderのインスタンスを作成し、リポジトリからドキュメントをロードする準備をする
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",  # クローンするGitリポジトリのURL
    repo_path="./langchain",  # ローカルにクローンするパス
    branch="master",  # チェックアウトするブランチ名
    file_filter=file_filter,  # 使用するファイルフィルタ関数
)

# ドキュメントをロードし、リストとして取得
documents = loader.load()
# ロードしたドキュメント数を出力
print(len(documents))

418


### Ragas による合成テストデータ生成の実装


In [8]:
# 各ドキュメントのメタデータに"filename"キーを追加し、"source"の値をコピーする
for document in documents:  # documentsリスト内の各ドキュメントに対して処理を行う
    document.metadata["filename"] = document.metadata["source"]  # "source"の値を"filename"キーに設定する

In [ ]:
import nest_asyncio  # Jupyter環境などで非同期処理の競合を防ぐためのパッケージ
from ragas.testset.generator import TestsetGenerator  # Ragasのテストセット生成クラスをインポート
from ragas.testset.evolutions import simple,reasoning,multi_context  # テストデータ生成時の進化タイプをインポート
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # OpenAIのチャットモデルと埋め込みモデルをインポート

# Jupyterなどの環境でasyncioのイベントループの競合を回避する
nest_asyncio.apply()

# テストセット生成器を初期化する
generator=TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),  # 質問生成用のLLMを指定
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),     # 評価用のLLMを指定
    embeddings=OpenAIEmbeddings(),                  # 埋め込みモデルを指定
)

# LangChainのドキュメントからテストセットを生成する
testset=generator.generate_with_langchain_docs(
    documents,  # テストデータ生成対象のドキュメント
    test_size=6,  # 生成するテストデータ数
    distributions={simple:0.5,reasoning:0.25,multi_context:0.25}  # 各進化タイプの割合を指定
)

embedding nodes:   0%|          | 0/1296 [00:00<?, ?it/s]

Generating:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# テストセットの中身確認
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of RAG evaluation in the c...,[# Vectorize\n\n> [Vectorize](https://vectoriz...,The answer to given question is not present in...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What is the significance of interoperability i...,[---\npagination_prev: null\npagination_next: ...,Interoperability in LangChain components is si...,simple,[{'source': 'docs/docs/contributing/how_to/int...,True
2,What is the purpose of Datadog in the context ...,[# Datadog Logs\n\n>[Datadog](https://www.data...,Datadog is a monitoring and analytics platform...,simple,[{'source': 'docs/docs/integrations/providers/...,True
3,What enables easy component swaps in LangChain...,[---\npagination_prev: null\npagination_next: ...,LangChain components expose a standard interfa...,reasoning,[{'source': 'docs/docs/contributing/how_to/int...,True
4,What’s needed for Banana's GPU inference?,[# Banana\n\n>[Banana](https://www.banana.dev/...,"To use Banana's GPU inference, you need to ins...",reasoning,[{'source': 'docs/docs/integrations/providers/...,True
5,How does LangChain's standard interface help d...,[---\npagination_prev: null\npagination_next: ...,LangChain's standard interface helps developer...,multi_context,[{'source': 'docs/docs/contributing/how_to/int...,True
6,How does langchain-robocorp aid Python workers...,[# Sema4 (fka Robocorp)\n\n>[Robocorp](https:/...,The context does not provide specific informat...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


### LangSmith の Dataset の作成


In [ ]:
from langsmith import Client  # LangSmithのClientクラスをインポートし、データセット管理を行う

# データセット名を定義
dataset_name = "agent-book"  # 作成・削除対象となるデータセット名を指定

# LangSmithクライアントのインスタンスを作成
client = Client()  # API操作用のクライアントを初期化

# 既存の同名データセットが存在する場合は削除する
if client.has_dataset(dataset_name=dataset_name):  # 指定した名前のデータセットが存在するか確認
    client.delete_dataset(dataset_name=dataset_name)  # 存在する場合はデータセットを削除

# 新しいデータセットを作成する
dataset = client.create_dataset(dataset_name=dataset_name)  # 指定した名前で新規データセットを作成し、変数に格納

# LangSmithでデータセットを確認する


### 合成テストデータの保存


In [ ]:
# 入力データ、出力データ、メタデータを格納するリストを初期化
inputs=[]  # 各テストセットの入力（質問）を格納するリスト
outputs=[]  # 各テストセットの出力（コンテキストと正解）を格納するリスト
metadatas=[]  # 各テストセットのメタデータ（ソースや進化タイプ）を格納するリスト

# テストセットの各レコードを処理し、必要な情報をリストに追加
for testset_record in testset.test_data:  # testsetの各テストデータを1件ずつ処理
	# 質問文をinputsリストに追加
	inputs.append(  # inputsリストに辞書を追加
		{
			"question":testset_record.question,  # 質問文を格納
		}
	)
	# コンテキストと正解をoutputsリストに追加
	outputs.append(  # outputsリストに辞書を追加
		{
			"contexts":testset_record.contexts,  # 関連コンテキストを格納
			"ground_truth":testset_record.ground_truth  # 正解（期待される回答）を格納
		}
	)
	# ソースと進化タイプをmetadatasリストに追加
	metadatas.append(  # metadatasリストに辞書を追加
		{
			"source":testset_record.metadata[0]["source"],  # ドキュメントのソース情報を格納
			"evolution_type":testset_record.evolution_type  # 進化タイプを格納
		}
	)

# 収集したinputs, outputs, metadatasを使ってLangSmithに例を一括登録
client.create_examples(  # LangSmithのデータセットに例を一括作成
	inputs=inputs,  # 入力データ（質問リスト）
	outputs=outputs,  # 出力データ（コンテキストと正解リスト）
	metadata=metadatas,  # メタデータ（ソース・進化タイプリスト）
	dataset_id=dataset.id  # 対象データセットのID
)

## 7.5. LangSmith と Ragas を使ったオフライン評価の実装


### カスタム Evaluator の実装


In [ ]:
from typing import Any  # typingモジュールからAny型をインポート。型ヒントで任意の型を指定するために使用。
from langchain_core.embeddings import Embeddings  # langchain_core.embeddingsからEmbeddingsクラスをインポート。埋め込みモデルの型指定用。
from langchain_core.language_models import BaseChatModel  # langchain_core.language_modelsからBaseChatModelをインポート。チャットモデルの基底クラス。
from langsmith.schemas import Example, Run  # langsmith.schemasからExampleとRunをインポート。評価対象の例と実行結果のスキーマ。
from ragas.embeddings import LangchainEmbeddingsWrapper  # ragas.embeddingsからLangchainEmbeddingsWrapperをインポート。LangChainの埋め込みをRagas用にラップ。
from ragas.llms import LangchainLLMWrapper  # ragas.llmsからLangchainLLMWrapperをインポート。LangChainのLLMをRagas用にラップ。
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM  # ragas.metrics.baseから評価指標の基底クラスと埋め込み/LLM対応クラスをインポート。

# Ragasの評価指標をLangSmith Evaluatorとして利用するためのラッパークラスを定義
class RagasMetricEvaluator:
    # 初期化メソッド。評価指標、LLM、埋め込みモデルを受け取る
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric  # 渡された評価指標をインスタンス変数に格納

        # 評価指標がLLMを必要とする場合、LLMラッパーを設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)  # LLMをRagas用ラッパーで包んでセット
        # 評価指標が埋め込みを必要とする場合、埋め込みラッパーを設定
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)  # 埋め込みをRagas用ラッパーで包んでセット

    # 評価メソッド。RunとExampleを受け取り、評価結果の辞書を返す
    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        # Runから文脈（contexts）のテキストを抽出しリスト化
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価指標のscoreメソッドを呼び出してスコアを算出
        score = self.metric.score(
            {  # 評価に必要な情報を辞書で渡す
                "question": example.inputs["question"],  # 質問文
                "answer": run.outputs["answer"],  # モデルの回答
                "contexts": context_strs,  # 関連文脈
                "ground_truth": example.outputs["ground_truth"],  # 正解
            },  # 各要素を評価指標に渡す
        )  # scoreメソッドでスコアを計算
        return {"key": self.metric.name, "score": score}  # 評価指標名とスコアを辞書で返す



from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # langchain_openaiからChatOpenAI（LLM）とOpenAIEmbeddings（埋め込みモデル）をインポート
from ragas.metrics import answer_relevancy, context_precision  # ragas.metricsからanswer_relevancyとcontext_precisionの評価指標をインポート

# 使用する評価指標をリストで定義
metrics = [context_precision, answer_relevancy]  # 文脈精度と回答関連性

# OpenAIのGPT-4oモデルをLLMとして初期化
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # gpt-4oモデルを温度0で生成

# OpenAIのtext-embedding-3-smallモデルで埋め込みモデルを初期化
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # text-embedding-3-smallモデルで埋め込み生成

# 各評価指標ごとにRagasMetricEvaluatorを生成し、evaluateメソッドをリスト化
evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate  # 各評価指標に対してEvaluatorを作成し、evaluateメソッドを取得
    for metric in metrics  # metricsリスト内の各指標に対して処理
]

### 推論の関数の実装


In [ ]:
from langchain_chroma import Chroma  # Chromaベースのベクトルストアを利用するためのクラスをインポート
from langchain_openai import OpenAIEmbeddings  # OpenAIの埋め込みモデルを利用するためのクラスをインポート

# OpenAIのtext-embedding-3-smallモデルを用いて埋め込みモデルを初期化
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # OpenAIEmbeddings: テキストをベクトル化するためのクラス

# ドキュメント群からChromaベクトルストアを作成
db = Chroma.from_documents(documents, embeddings)  # Chroma: ベクトルストア、from_documentsで初期化

from langchain_core.output_parsers import StrOutputParser  # StrOutputParser: LLM出力を文字列として扱う
from langchain_core.prompts import ChatPromptTemplate  # ChatPromptTemplate: チャット用プロンプトのテンプレート化
from langchain_core.runnables import RunnableParallel, RunnablePassthrough  # RunnableParallel: 並列実行, RunnablePassthrough: 入力をそのまま渡す
from langchain_openai import ChatOpenAI  # ChatOpenAI: OpenAIのチャットLLMを利用

# プロンプトテンプレートを定義（複数行）
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
文脈: """
{context}
"""
質問: {question}
''')  # テンプレート定義終了

# gpt-4o-miniモデルを用いてLLMを初期化
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # ChatOpenAI: gpt-4o-miniモデルを温度0で生成

# ベクトルストアからレトリバーを作成
retriever = db.as_retriever()  # as_retriever: ベクトル検索用のレトリバーを取得

# 推論チェーンを構築（複数行）
chain = RunnableParallel(  # RunnableParallel: 並列に複数の処理を実行するチェーンを作成
    {
        "question": RunnablePassthrough(),  # "question"キーはそのまま渡す
        "context": retriever,  # "context"キーにはレトリバーの出力を渡す
    }
).assign(answer=prompt | model | StrOutputParser())  # assignで"answer"キーにプロンプト→モデル→出力パーサの流れを設定

### オフライン評価の実装・実行


In [ ]:
# 入力辞書から質問を受け取り、推論チェーンを実行して結果を返す関数を定義
def predict(inputs: dict[str, Any]) -> dict[str, Any]:  # 入力と出力の型アノテーション付きで関数を定義
    # 入力辞書から"question"キーの値（質問文）を取得
    question = inputs["question"]
    # チェーンに質問を渡して推論を実行し、出力を取得
    output = chain.invoke(question)
    # 推論結果から文脈と回答を辞書形式で返す
    return {
        "contexts": output["context"],  # 取得した文脈を"contexts"キーに格納
        "answer": output["answer"],     # 生成された回答を"answer"キーに格納
    }


from langsmith.evaluation import evaluate  # LangSmithの評価用関数をインポート

# オフライン評価を実行する。predict関数を評価対象とし、データセットと評価指標を指定
evaluate(
    predict,            # 評価対象の関数
    data="agent-book",  # 使用するデータセット名
    evaluators=evaluators,  # 使用する評価指標
)

# 評価結果の比較ページURL（参考用）
# https://smith.langchain.com/o/bebc8bb1-8ebf-49c3-8c20-fe0c2220db92/datasets/a5328057-4fa4-4330-a4c2-a7e4d3135c2f/compare?selectedSessions=64b1006f-bbb9-474f-ac13-c5aedf271a8d


View the evaluation results for experiment: 'crazy-history-69' at:
https://smith.langchain.com/o/bebc8bb1-8ebf-49c3-8c20-fe0c2220db92/datasets/a5328057-4fa4-4330-a4c2-a7e4d3135c2f/compare?selectedSessions=61f77f30-69d0-41c3-9f6b-7f9a8736b5e4




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator evaluate> on run cff735a0-bf7b-4722-b8e0-aeecf0952fe5: APIConnectionError('Connection error.')
Traceback (most recent call last):
  File "/Users/kenichi/Projects/rag_ai_agent_book/.venv/lib/python3.13/site-packages/openai/_base_client.py", line 1548, in _request
    response = await self._client.send(
               ^^^^^^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
    )
    ^
  File "/Users/kenichi/Projects/rag_ai_agent_book/.venv/lib/python3.13/site-packages/httpx/_client.py", line 1674, in send
    response = await self._send_handling_auth(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<4 lines>...
    )
    ^
  File "/Users/kenichi/Projects/rag_ai_agent_book/.venv/lib/python3.13/site-packages/httpx/_client.py", line 1702, in _send_handling_auth
    response = await self._send_handling_redirects(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
    )
    ^
  File "/Users/kenichi/Projects/rag_ai_agent_book/.venv/l

## LangSmith を使ったオンライン評価の実装


### フィードバックボタンを表示する関数の実装


In [ ]:
from uuid import UUID  # UUID型をインポート
import ipywidgets as widgets  # ipywidgetsのwidgetsモジュールをインポート
from IPython.display import display  # display関数をインポート
from langsmith import Client  # LangSmithのClientクラスをインポート

# フィードバック用のボタンを表示する関数を定義
def display_feedback_buttons(run_id:UUID)->None:
	# 「Good」ボタンを作成
	good_button=widgets.Button(
		description="Good",  # ボタンのラベル
		button_style="success",  # ボタンのスタイル
		icon="thumbs-up"  # ボタンのアイコン
	)
	# 「Bad」ボタンを作成
	bad_button=widgets.Button(
		description="Bad",  # ボタンのラベル
		button_style="danger",  # ボタンのスタイル
		icon="thumbs-down"  # ボタンのアイコン
	)

	# ボタンクリック時のコールバック関数を定義
	def on_button_clicked(button:widgets.Button)->None:
		# クリックされたボタンがgood_buttonの場合
		if button==good_button:
			score=1  # スコアを1に設定
		# クリックされたボタンがbad_buttonの場合
		elif button==bad_button:
			score=0  # スコアを0に設定
		# それ以外の場合はエラー
		else:
			raise ValueError(f"Unknown button:{button}")  # 未知のボタンエラー

		# LangSmithクライアントを作成
		client=Client()
		# フィードバックを送信
		client.create_feedback(run_id=run_id, key="thumbs",score=score)
		# 送信完了メッセージを表示
		print("フィードバックを送信しました")

	# good_buttonにクリックイベントを登録
	good_button.on_click(on_button_clicked)
	# bad_buttonにクリックイベントを登録
	bad_button.on_click(on_button_clicked)

	# 2つのボタンを表示
	display(good_button, bad_button)


### フィードバックボタンを表示


In [ ]:
from langchain_core.tracers.context import collect_runs  # collect_runsをインポート

# LangChainの実行をトレースし、出力とrun_idを取得
with collect_runs() as runs_cb:
	output=chain.invoke("LangChainの概要を教えて")  # チェーンに質問を投げて出力を取得
	print(output["answer"])  # 回答部分のみ表示
	run_id=runs_cb.traced_runs[0].id  # 最初のrun_idを取得

# フィードバックボタンを表示
display_feedback_buttons(run_id)

LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下の3つの主要なステージがあります。

1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築します。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。

2. **プロダクショナリゼーション**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできるようにします。

3. **デプロイメント**: LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換します。

LangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、さまざまな統合パッケージやコミュニティによって維持されるコンポーネントが含まれています。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())